In [ ]:
import yfinance as yf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
pd.options.display.max_columns = None

In [ ]:
ticker = 'AAPL'
stock = yf.Ticker(ticker)

# get required data
income_stmt = stock.quarterly_financials.T.sort_index()
balance_sheet = stock.quarterly_balance_sheet.T.sort_index()
cashflow_stmt = stock.quarterly_cashflow.T.sort_index()
info = stock.info

In [ ]:
income_stmt.isnull().sum()

In [ ]:
balance_sheet.drop('2023-12-31', inplace= True)

In [ ]:
'Total Liab' in balance_sheet.columns

In [ ]:
balance_sheet.columns

In [ ]:
cashflow_stmt

In [ ]:
quarters = income_stmt.index.intersection(balance_sheet.index).intersection(cashflow_stmt.index)

quarters

In [ ]:
ratio_df = pd.DataFrame(index=quarters)

In [ ]:
ratio_df['Quarter'] = quarters.to_series().apply(lambda x:((x.month - 1) // 3) + 1)

In [ ]:
ratio_df['Year'] = quarters.to_series().apply(lambda x:x.year)

In [ ]:
ratio_df

In [ ]:
latest_date = quarters[-1]

for date in quarters:
    try:
        # --- Financials ---
        revenue = income_stmt.loc[date, "Total Revenue"]
        net_income = income_stmt.loc[date, "Net Income"]
        gross_profit = income_stmt.loc[date, "Gross Profit"]
        operating_income = income_stmt.loc[date, "Operating Income"]

        total_assets = balance_sheet.loc[date, "Total Assets"]
        total_liabilities = balance_sheet.loc[date, "Total Liabilities Net Minority Interest"]
        equity = balance_sheet.loc[date, "Stockholders Equity"]
        current_assets = balance_sheet.loc[date, "Current Assets"]
        current_liabilities = balance_sheet.loc[date, "Current Liabilities"]
        inventory = balance_sheet.get("Inventory", pd.Series([0])).get(date, 0)
        cash = balance_sheet.get("Cash And Cash Equivalents", pd.Series([0])).get(date, 0)
        receivables = balance_sheet.get("Accounts Receivable", pd.Series([0])).get(date, 0)
        invested_capital = balance_sheet.get("Invested Capital", pd.Series([np.nan])).get(date)
        retained_earnings = balance_sheet.get("Retained Earnings", pd.Series([np.nan])).get(date)
        ebit = income_stmt.get("EBIT", income_stmt.get("Operating Income", pd.Series([np.nan]))).get(date)

        op_cash_flow = cashflow_stmt.loc[date, "Operating Cash Flow"]
        capex = cashflow_stmt.get("Capital Expenditures", pd.Series([0])).get(date, 0)
        free_cash_flow = op_cash_flow - capex
        working_capital = current_assets - current_liabilities

        # --- Ratios ---
        ratio_df.loc[date, "Net Profit Margin"] = net_income / revenue
        ratio_df.loc[date, "Gross Margin"] = gross_profit / revenue
        ratio_df.loc[date, "Operating Margin"] = operating_income / revenue
        ratio_df.loc[date, "ROA"] = net_income / total_assets
        ratio_df.loc[date, "ROE"] = net_income / equity
        ratio_df.loc[date, "Current Ratio"] = current_assets / current_liabilities
        ratio_df.loc[date, "Quick Ratio"] = (current_assets - inventory) / current_liabilities
        ratio_df.loc[date, "Cash Ratio"] = cash / current_liabilities
        ratio_df.loc[date, "Debt-to-Equity"] = total_liabilities / equity
        ratio_df.loc[date, "Debt Ratio"] = total_liabilities / total_assets
        ratio_df.loc[date, "Free Cash Flow"] = free_cash_flow
        ratio_df.loc[date, "Cash Flow Margin"] = op_cash_flow / revenue
        ratio_df.loc[date, "Inventory Turnover"] = revenue / inventory if inventory else np.nan
        ratio_df.loc[date, "Asset Turnover"] = revenue / total_assets
        ratio_df.loc[date, "Receivables Turnover"] = revenue / receivables if receivables else np.nan
        ratio_df.loc[date, "CapEx Intensity"] = capex / revenue if revenue else np.nan
        ratio_df.loc[date, "ROCE"] = operating_income / invested_capital if invested_capital else np.nan
        ratio_df.loc[date, "FCF Conversion"] = free_cash_flow / net_income if net_income else np.nan

        # --- Altman Z-Score ---
        market_cap = info.get("marketCap")
        if all(pd.notnull([working_capital, total_assets, retained_earnings, ebit, market_cap, total_liabilities, revenue])):
            A = working_capital / total_assets
            B = retained_earnings / total_assets
            C = ebit / total_assets
            D = market_cap / total_liabilities
            E = revenue / total_assets
            z_score = 1.2 * A + 1.4 * B + 3.3 * C + 0.6 * D + 1.0 * E
            ratio_df.loc[date, "Altman Z-Score"] = z_score
        else:
            ratio_df.loc[date, "Altman Z-Score"] = np.nan

        # --- PEG, P/B, Interest Coverage (only for latest quarter) ---
        if date == latest_date:
            trailing_pe = info.get("trailingPE", None)
            eps_growth = info.get("earningsGrowth", None)  # Usually in decimal (e.g. 0.12)
            peg_ratio = trailing_pe / (eps_growth * 100) if trailing_pe and eps_growth else np.nan
            ratio_df.loc[date, "PEG Ratio"] = peg_ratio

            interest_expense = income_stmt.get("Interest Expense", pd.Series([np.nan])).get(date)
            interest_coverage = ebit / abs(interest_expense) if interest_expense and interest_expense != 0 else np.nan
            ratio_df.loc[date, "Interest Coverage"] = interest_coverage

            current_price = info.get("currentPrice")
            shares_outstanding = info.get("sharesOutstanding")
            book_value_per_share = equity / shares_outstanding if shares_outstanding else np.nan
            pb_ratio = current_price / book_value_per_share if book_value_per_share else np.nan
            ratio_df.loc[date, "Price-to-Book"] = pb_ratio

    except Exception as e:
        print(f"[{date}] Skipped due to error: {e}")


In [ ]:
ratio_df

In [26]:
import yfinance as yf
import numpy as np
import pandas as pd

class stock():
    def __init__(self, ticker):
        # Intilializing Data Points
        self.ticker = ticker
        self.stock = yf.Ticker(self.ticker)
        
        # Getting Quarterly Data from yfinance
        self.q_income_stmt = self.stock.quarterly_financials.T.sort_index()
        self.q_balance_sheet = self.stock.quarterly_balance_sheet.T.sort_index()
        self.q_cashflow_stmt = self.stock.quarterly_cashflow.T.sort_index()
        self.info = self.stock.info

        # Getting yearly Data from yfinance
        self.y_income_stmt = self.stock.financials.T.sort_index()
        self.y_balance_sheet = self.stock.balance_sheet.T.sort_index()
        self.y_cashflow_stmt = self.stock.cashflow.T.sort_index()
        self.info = self.stock.info
        
        # Calculating Some Basic Things
        self.q_dates = self.q_income_stmt.index.intersection(self.q_balance_sheet.index).intersection(self.q_cashflow_stmt.index)
        self.y_dates = self.y_income_stmt.index.intersection(self.y_balance_sheet.index).intersection(self.y_cashflow_stmt.index)
        self.qratios = pd.DataFrame(index=self.q_dates)
        self.qfinancials = pd.DataFrame(index=self.q_dates)
        self.qratios['Quarter'] = self.q_dates.to_series().apply(lambda x:((x.month - 1) // 3) + 1)
        self.qratios['Year'] = self.q_dates.to_series().apply(lambda x:x.year)
        self.qfinancials['Quarter'] = self.q_dates.to_series().apply(lambda x:((x.month - 1) // 3) + 1)
        self.qfinancials['Year'] = self.q_dates.to_series().apply(lambda x:x.year)
        self.latest_quarter = self.q_dates[-1]
        self.latest_year = self.y_dates[-1]
    
    def get_safe_value(self, df, key, date, default=np.nan):
        series = df.get(key)
        return series.get(date) if series is not None else default

    def calculate_quarterly_ratios(self):
        for date in self.q_dates:
            try:
                # Calculating Financials
                revenue = self.q_income_stmt.loc[date, "Total Revenue"]
                net_income = self.q_income_stmt.loc[date, "Net Income"]
                gross_profit = self.q_income_stmt.loc[date, "Gross Profit"]
                operating_income = self.q_income_stmt.loc[date, "Operating Income"]
                total_assets = self.q_balance_sheet.loc[date, "Total Assets"]
                total_liabilities = self.q_balance_sheet.loc[date, "Total Liabilities Net Minority Interest"]
                equity = self.q_balance_sheet.loc[date, "Stockholders Equity"]
                current_assets = self.q_balance_sheet.loc[date, "Current Assets"]
                current_liabilities = self.q_balance_sheet.loc[date, "Current Liabilities"]
                
                inventory = self.get_safe_value(self.q_balance_sheet, "Inventory", date, default=np.nan)
                cash = self.get_safe_value(self.q_balance_sheet, "Cash And Cash Equivalents", date, default=np.nan)
                receivables = self.get_safe_value(self.q_balance_sheet, "Accounts Receivable", date, default=np.nan)
                invested_capital = self.get_safe_value(self.q_balance_sheet, "Invested Capital", date, default=np.nan)
                retained_earnings = self.get_safe_value(self.q_balance_sheet, "Retained Earnings", date, default=np.nan)
                ebit = self.get_safe_value(self.q_income_stmt, "EBIT", date, default=np.nan)
                if pd.isna(ebit):
                    ebit = self.get_safe_value(self.q_income_stmt, "Operating Income", date, default=np.nan)

                op_cash_flow = self.q_cashflow_stmt.loc[date, "Operating Cash Flow"]
                capex = self.get_safe_value(self.q_cashflow_stmt, "Capital Expenditure", date, default=0)
                free_cash_flow = op_cash_flow - capex
                working_capital = current_assets - current_liabilities

                # Storing the above Financials
                self.qfinancials.loc[date, "Revenue"] = revenue
                self.qfinancials.loc[date, "Net Income"] = net_income
                self.qfinancials.loc[date, "Gross Profit"] = gross_profit
                self.qfinancials.loc[date, "Operating Income"] = operating_income

                self.qfinancials.loc[date, "Total Assets"] = total_assets
                self.qfinancials.loc[date, "Total Liabilities"] = total_liabilities
                self.qfinancials.loc[date, "Equity"] = equity
                self.qfinancials.loc[date, "Current Assets"] = current_assets
                self.qfinancials.loc[date, "Current Liabilities"] = current_liabilities
                self.qfinancials.loc[date, "Inventory"] = inventory
                self.qfinancials.loc[date, "Cash"] = cash
                self.qfinancials.loc[date, "Receivables"] = receivables
                self.qfinancials.loc[date, "Invested Capital"] = invested_capital
                self.qfinancials.loc[date, "Retained Earnings"] = retained_earnings
                self.qfinancials.loc[date, "EBIT"] = ebit
                self.qfinancials.loc[date, "Free Cash Flow"] = free_cash_flow

                self.qfinancials.loc[date, "Operating Cash Flow"] = op_cash_flow
                self.qfinancials.loc[date, "Capital Expenditure"] = capex
                self.qfinancials.loc[date, "Free Cash Flow"] = free_cash_flow
                self.qfinancials.loc[date, "Working Capital"] = working_capital

                # Calculating and storing Financials
                self.qratios.loc[date, "Net Profit Margin"] = net_income / revenue
                self.qratios.loc[date, "Gross Margin"] = gross_profit / revenue
                self.qratios.loc[date, "Operating Margin"] = operating_income / revenue
                self.qratios.loc[date, "ROA"] = net_income / total_assets
                self.qratios.loc[date, "ROE"] = net_income / equity
                self.qratios.loc[date, "Current Ratio"] = current_assets / current_liabilities
                self.qratios.loc[date, "Quick Ratio"] = (current_assets - inventory) / current_liabilities
                self.qratios.loc[date, "Cash Ratio"] = cash / current_liabilities
                self.qratios.loc[date, "Debt-to-Equity"] = total_liabilities / equity
                self.qratios.loc[date, "Debt Ratio"] = total_liabilities / total_assets
                self.qratios.loc[date, "Cash Flow Margin"] = op_cash_flow / revenue
                self.qratios.loc[date, "Inventory Turnover"] = revenue / inventory if inventory else np.nan
                self.qratios.loc[date, "Asset Turnover"] = revenue / total_assets
                self.qratios.loc[date, "Receivables Turnover"] = revenue / receivables if receivables else np.nan
                self.qratios.loc[date, "CapEx Intensity"] = capex / revenue if revenue else np.nan
                self.qratios.loc[date, "ROCE"] = operating_income / invested_capital if invested_capital else np.nan
                self.qratios.loc[date, "FCF Conversion"] = free_cash_flow / net_income if net_income else np.nan

                # Altman-Z Score
                market_cap = self.info.get("marketCap")
                if all(pd.notnull([working_capital, total_assets, retained_earnings, ebit, market_cap, total_liabilities, revenue])):
                    A = working_capital / total_assets
                    B = retained_earnings / total_assets
                    C = ebit / total_assets
                    D = market_cap / total_liabilities
                    E = revenue / total_assets
                    z_score = 1.2 * A + 1.4 * B + 3.3 * C + 0.6 * D + 1.0 * E
                    self.qratios.loc[date, "Altman Z-Score"] = z_score
                else:
                    self.qratios.loc[date, "Altman Z-Score"] = np.nan

                # PEG, P/B, Integerst Coverage for Last Quarter
                if date == self.latest_quarter:
                    trailing_pe = self.info.get("trailingPE", None)
                    eps_growth = self.info.get("earningsGrowth", None)  # Usually in decimal (e.g. 0.12)
                    print(trailing_pe, eps_growth)
                    self.peg_ratio = trailing_pe / (eps_growth * 100) if trailing_pe and eps_growth and eps_growth > 0 else "N/A"

                    
                    # Data Not Available
                    # interest_expense = self.q_income_stmt.get("Interest Expense", pd.Series([np.nan])).get(date)
                    # interest_coverage = ebit / abs(interest_expense) if interest_expense and interest_expense != 0 else np.nan
                    # self.ratios.loc[date, "Interest Coverage"] = interest_coverage

                    current_price = self.info.get("currentPrice")
                    shares_outstanding = self.info.get("sharesOutstanding")
                    book_value_per_share = equity / shares_outstanding if shares_outstanding else np.nan
                    self.pb_ratio = current_price / book_value_per_share if book_value_per_share else np.nan

                    growth_cols = [ "Revenue", "Net Income", "Gross Profit", "Operating Income", "Operating Cash Flow", "Free Cash Flow", "EBIT" ]

                    for col in growth_cols:
                        if col in self.financials.columns:
                            self.financials[f"{col} QoQ"] = self.financials[col].pct_change().round(4) * 100

            except Exception as e:
                print(f"[{date}] Skipped due to error: {e}")
        
        self.format_ratios(type = 'q')
        self.qfinancials.dropna()
        self.qratios.dropna()

    def calculate_yearly_ratios(self):
        self.yratios = pd.DataFrame(index=self.y_dates)
        self.yfinancials = pd.DataFrame(index=self.y_dates)
        self.yfinancials['Year'] = self.y_dates.to_series().apply(lambda x: x.year)

        for date in self.y_dates:
            try:
                # Pull values from yearly financials
                revenue = self.y_income_stmt.loc[date, "Total Revenue"]
                net_income = self.y_income_stmt.loc[date, "Net Income"]
                gross_profit = self.y_income_stmt.loc[date, "Gross Profit"]
                operating_income = self.y_income_stmt.loc[date, "Operating Income"]
                total_assets = self.y_balance_sheet.loc[date, "Total Assets"]
                total_liabilities = self.y_balance_sheet.loc[date, "Total Liabilities Net Minority Interest"]
                equity = self.y_balance_sheet.loc[date, "Stockholders Equity"]
                current_assets = self.y_balance_sheet.loc[date, "Current Assets"]
                current_liabilities = self.y_balance_sheet.loc[date, "Current Liabilities"]

                inventory = self.get_safe_value(self.y_balance_sheet, "Inventory", date)
                cash = self.get_safe_value(self.y_balance_sheet, "Cash And Cash Equivalents", date)
                receivables = self.get_safe_value(self.y_balance_sheet, "Accounts Receivable", date)
                invested_capital = self.get_safe_value(self.y_balance_sheet, "Invested Capital", date)
                retained_earnings = self.get_safe_value(self.y_balance_sheet, "Retained Earnings", date)
                ebit = self.get_safe_value(self.y_income_stmt, "EBIT", date)
                if pd.isna(ebit):
                    ebit = self.get_safe_value(self.y_income_stmt, "Operating Income", date)

                op_cash_flow = self.y_cashflow_stmt.loc[date, "Operating Cash Flow"]
                capex = self.get_safe_value(self.y_cashflow_stmt, "Capital Expenditure", date, default=0)
                free_cash_flow = op_cash_flow - capex
                working_capital = current_assets - current_liabilities

                # Store financials
                self.yfinancials.loc[date, "Revenue"] = revenue
                self.yfinancials.loc[date, "Net Income"] = net_income
                self.yfinancials.loc[date, "Gross Profit"] = gross_profit
                self.yfinancials.loc[date, "Operating Income"] = operating_income
                self.yfinancials.loc[date, "Total Assets"] = total_assets
                self.yfinancials.loc[date, "Total Liabilities"] = total_liabilities
                self.yfinancials.loc[date, "Equity"] = equity
                self.yfinancials.loc[date, "Current Assets"] = current_assets
                self.yfinancials.loc[date, "Current Liabilities"] = current_liabilities
                self.yfinancials.loc[date, "Inventory"] = inventory
                self.yfinancials.loc[date, "Cash"] = cash
                self.yfinancials.loc[date, "Receivables"] = receivables
                self.yfinancials.loc[date, "Invested Capital"] = invested_capital
                self.yfinancials.loc[date, "Retained Earnings"] = retained_earnings
                self.yfinancials.loc[date, "EBIT"] = ebit
                self.yfinancials.loc[date, "Operating Cash Flow"] = op_cash_flow
                self.yfinancials.loc[date, "Capital Expenditure"] = capex
                self.yfinancials.loc[date, "Free Cash Flow"] = free_cash_flow
                self.yfinancials.loc[date, "Working Capital"] = working_capital

                # Ratios
                self.yratios.loc[date, "Net Profit Margin"] = net_income / revenue
                self.yratios.loc[date, "Gross Margin"] = gross_profit / revenue
                self.yratios.loc[date, "Operating Margin"] = operating_income / revenue
                self.yratios.loc[date, "ROA"] = net_income / total_assets
                self.yratios.loc[date, "ROE"] = net_income / equity
                self.yratios.loc[date, "Current Ratio"] = current_assets / current_liabilities
                self.yratios.loc[date, "Quick Ratio"] = (current_assets - inventory) / current_liabilities
                self.yratios.loc[date, "Cash Ratio"] = cash / current_liabilities
                self.yratios.loc[date, "Debt-to-Equity"] = total_liabilities / equity
                self.yratios.loc[date, "Debt Ratio"] = total_liabilities / total_assets
                self.yratios.loc[date, "Cash Flow Margin"] = op_cash_flow / revenue
                self.yratios.loc[date, "Inventory Turnover"] = revenue / inventory if inventory else np.nan
                self.yratios.loc[date, "Asset Turnover"] = revenue / total_assets
                self.yratios.loc[date, "Receivables Turnover"] = revenue / receivables if receivables else np.nan
                self.yratios.loc[date, "CapEx Intensity"] = capex / revenue if revenue else np.nan
                self.yratios.loc[date, "ROCE"] = operating_income / invested_capital if invested_capital else np.nan
                self.yratios.loc[date, "FCF Conversion"] = free_cash_flow / net_income if net_income else np.nan

            except Exception as e:
                print(f"[Yearly {date}] Skipped due to error: {e}")

        # Optional: format like quarterly
        self.format_ratios()
        self.yfinancials.dropna()
        self.yratios.dropna()

    def format_ratios(self, type):
        percent_columns = [
            "Net Profit Margin", "Gross Margin", "Operating Margin",
            "ROA", "ROE", "Cash Flow Margin", "CapEx Intensity",
            "FCF Conversion"
        ]

        ratio_columns = [
            "Current Ratio", "Quick Ratio", "Cash Ratio",
            "Debt-to-Equity", "Debt Ratio", "Inventory Turnover",
            "Asset Turnover", "Receivables Turnover",
            "ROCE", "Altman Z-Score", "PEG Ratio", "Price-to-Book"
        ]

        if type == 'q':
            for col in percent_columns:
                if col in self.qratios.columns:
                    self.qratios[col] = self.qratios[col].round(5)

            for col in ratio_columns:
                if col in self.qratios.columns:
                    self.qratios[col] = self.qratios[col].round(4)

In [ ]:
stock = stock('AAPL')
stock.calculate_quarterly_ratios()
stock.yfinancials

33.37676 0.078
[2025-03-31 00:00:00] Skipped due to error: 'stock' object has no attribute 'financials'


,Quarter,Year,Revenue,Net Income,Gross Profit,Operating Income,Total Assets,Total Liabilities,Equity,Current Assets,Current Liabilities,Inventory,Cash,Receivables,Invested Capital,Retained Earnings,EBIT,Free Cash Flow,Operating Cash Flow,Capital Expenditure,Working Capital
2024-03-31,1,2024,9.075300e+10,2.363600e+10,4.227100e+10,2.790000e+10,3.374110e+11,2.632170e+11,7.419400e+10,1.284160e+11,1.238220e+11,6.232000e+09,3.269500e+10,2.183700e+10,1.787840e+11,4.339000e+09,2.790000e+10,2.468600e+10,2.269000e+10,-1.996000e+09,4.594000e+09
2024-06-30,2,2024,8.577700e+10,2.144800e+10,3.967800e+10,2.535200e+10,3.316120e+11,2.649040e+11,6.670800e+10,1.254350e+11,1.316240e+11,6.165000e+09,2.556500e+10,2.279500e+10,1.680120e+11,-4.726000e+09,2.535200e+10,3.100900e+10,2.885800e+10,-2.151000e+09,-6.189000e+09
2024-09-30,3,2024,9.493000e+10,1.473600e+10,4.387900e+10,2.959100e+10,3.649800e+11,3.080300e+11,5.695000e+10,1.529870e+11,1.763920e+11,7.286000e+09,2.994300e+10,3.341000e+10,1.635790e+11,-1.915400e+10,2.959100e+10,2.971900e+10,2.681100e+10,-2.908000e+09,-2.340500e+10
2024-12-31,4,2024,1.243000e+11,3.633000e+10,5.827500e+10,4.283200e+10,3.440850e+11,2.773270e+11,6.675800e+10,1.332400e+11,1.443650e+11,6.911000e+09,3.029900e+10,2.963900e+10,1.635570e+11,-1.122100e+10,4.283200e+10,3.287500e+10,2.993500e+10,-2.940000e+09,-1.112500e+10
2025-03-31,1,2025,9.535900e+10,2.478000e+10,4.486700e+10,2.958900e+10,3.312330e+11,2.644370e+11,6.679600e+10,1.186740e+11,1.445710e+11,6.269000e+09,2.816200e+10,2.613600e+10,1.649820e+11,-1.555200e+10,2.958900e+10,2.702300e+10,2.395200e+10,-3.071000e+09,-2.589700e+10


In [7]:
pd.options.display.max_columns = None

In [85]:
stock.ratios

,Quarter,Year,Net Profit Margin,Gross Margin,Operating Margin,ROA,ROE,Current Ratio,Quick Ratio,Cash Ratio,Debt-to-Equity,Debt Ratio,Cash Flow Margin,Inventory Turnover,Asset Turnover,Receivables Turnover,CapEx Intensity,ROCE,FCF Conversion,Altman Z-Score,PEG Ratio,Price-to-Book
2024-03-31,1,2024,0.260443,0.465781,0.307428,0.070051,0.318570,1.037102,0.986771,0.264048,3.547686,0.780108,0.250019,14.562420,0.268969,4.155928,-0.021994,0.156054,1.044424,7.853850,NaN,NaN
2024-06-30,2,2024,0.250044,0.462572,0.295557,0.064678,0.321521,0.952980,0.906142,0.194227,3.971098,0.798837,0.336431,13.913544,0.258667,3.762974,-0.025077,0.150894,1.445776,7.699927,NaN,NaN
2024-09-30,3,2024,0.155230,0.462225,0.311714,0.040375,0.258753,0.867313,0.826007,0.169753,5.408780,0.843964,0.282429,13.029097,0.260096,2.841365,-0.030633,0.180897,2.016762,6.596116,NaN,NaN
2024-12-31,4,2024,0.292277,0.468825,0.344586,0.105584,0.544204,0.922938,0.875067,0.209878,4.154214,0.805984,0.240829,17.985820,0.361248,4.193799,-0.023652,0.261878,0.904900,7.594971,NaN,NaN
2025-03-31,1,2025,0.259860,0.470506,0.310291,0.074811,0.370980,0.820870,0.777507,0.194797,3.958875,0.798341,0.251177,15.211198,0.287891,3.648569,-0.032205,0.179347,1.090517,7.667218,4.275371,47.797422


In [11]:
import yfinance as yf
import time

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"] * 10  # ~50 requests
failures = 0

for i, t in enumerate(tickers):
    try:
        print(f"[{i+1}] Fetching {t}...")
        info = yf.Ticker(t).info
        if not info:
            print(f"⚠️ {t}: Empty info (may be throttled)")
            failures += 1
        time.sleep(1)  # Delay to avoid rate limits
    except Exception as e:
        print(f"❌ {t}: {str(e)}")
        failures += 1

print(f"\nTotal Failures: {failures} / {len(tickers)}")


[1] Fetching AAPL...
❌ AAPL: Failed to perform, curl: (56) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[2] Fetching MSFT...
[3] Fetching GOOGL...
[4] Fetching AMZN...
[5] Fetching TSLA...
[6] Fetching AAPL...
[7] Fetching MSFT...
[8] Fetching GOOGL...
[9] Fetching AMZN...
[10] Fetching TSLA...
[11] Fetching AAPL...
[12] Fetching MSFT...
[13] Fetching GOOGL...
[14] Fetching AMZN...
[15] Fetching TSLA...
[16] Fetching AAPL...
[17] Fetching MSFT...
[18] Fetching GOOGL...
[19] Fetching AMZN...
[20] Fetching TSLA...
[21] Fetching AAPL...
[22] Fetching MSFT...
[23] Fetching GOOGL...
[24] Fetching AMZN...
[25] Fetching TSLA...
[26] Fetching AAPL...
[27] Fetching MSFT...
[28] Fetching GOOGL...


KeyboardInterrupt: 

In [86]:
def scale_data(df):
    columns = [x for x in df.columns if x not in ['Year', 'Quarter']]
    new_df = df.copy()

    for column in columns:
        max_val = df[column].abs().max()
        print(column, max_val)
        if pd.isna(max_val):
            scale = 1
            suffix = ''
        elif max_val >= 1e12:
            scale = 1e12
            suffix = "(Trillions)"
        elif max_val >= 1e9:
            scale = 1e9
            suffix = "(Billions)"
        elif max_val >= 1e6:
            scale = 1e6
            suffix = "(Millions)"
        else:
            scale = 1
            suffix = ''
        
        if suffix != '':
            new_col_name = f"{column} {suffix}"
            new_df[new_col_name] = df[column] / scale
            new_df.drop(column, axis = 1, inplace = True)
        

    return new_df

In [87]:
scale_data(stock.financials)

Revenue 124300000000.0
Net Income 36330000000.0
Gross Profit 58275000000.0
Operating Income 42832000000.0
Total Assets 364980000000.0
Total Liabilities 308030000000.0
Equity 74194000000.0
Current Assets 152987000000.0
Current Liabilities 176392000000.0
Inventory 7286000000.0
Cash 32695000000.0
Receivables 33410000000.0
Invested Capital 178784000000.0
Retained Earnings 19154000000.0
EBIT 42832000000.0
Free Cash Flow 32875000000.0
Operating Cash Flow 29935000000.0
Capital Expenditure 3071000000.0
Working Capital 25897000000.0


,Quarter,Year,Revenue (Billions),Net Income (Billions),Gross Profit (Billions),Operating Income (Billions),Total Assets (Billions),Total Liabilities (Billions),Equity (Billions),Current Assets (Billions),Current Liabilities (Billions),Inventory (Billions),Cash (Billions),Receivables (Billions),Invested Capital (Billions),Retained Earnings (Billions),EBIT (Billions),Free Cash Flow (Billions),Operating Cash Flow (Billions),Capital Expenditure (Billions),Working Capital (Billions)
2024-03-31,1,2024,90.753,23.636,42.271,27.900,337.411,263.217,74.194,128.416,123.822,6.232,32.695,21.837,178.784,4.339,27.900,24.686,22.690,-1.996,4.594
2024-06-30,2,2024,85.777,21.448,39.678,25.352,331.612,264.904,66.708,125.435,131.624,6.165,25.565,22.795,168.012,-4.726,25.352,31.009,28.858,-2.151,-6.189
2024-09-30,3,2024,94.930,14.736,43.879,29.591,364.980,308.030,56.950,152.987,176.392,7.286,29.943,33.410,163.579,-19.154,29.591,29.719,26.811,-2.908,-23.405
2024-12-31,4,2024,124.300,36.330,58.275,42.832,344.085,277.327,66.758,133.240,144.365,6.911,30.299,29.639,163.557,-11.221,42.832,32.875,29.935,-2.940,-11.125
2025-03-31,1,2025,95.359,24.780,44.867,29.589,331.233,264.437,66.796,118.674,144.571,6.269,28.162,26.136,164.982,-15.552,29.589,27.023,23.952,-3.071,-25.897


In [90]:
def format_ratios(df):
    percent_columns = [
        "Net Profit Margin", "Gross Margin", "Operating Margin",
        "ROA", "ROE", "Cash Flow Margin", "CapEx Intensity",
        "FCF Conversion"
    ]

    ratio_columns = [
        "Current Ratio", "Quick Ratio", "Cash Ratio",
        "Debt-to-Equity", "Debt Ratio", "Inventory Turnover",
        "Asset Turnover", "Receivables Turnover",
        "ROCE", "Altman Z-Score", "PEG Ratio", "Price-to-Book"
    ]

    for col in percent_columns:
        if col in df.columns:
            df[col] = df[col].round(5)

    for col in ratio_columns:
        if col in df.columns:
            df[col] = df[col].round(4)

    return df

In [91]:
format_ratios(stock.ratios)

,Quarter,Year,Net Profit Margin,Gross Margin,Operating Margin,ROA,ROE,Current Ratio,Quick Ratio,Cash Ratio,Debt-to-Equity,Debt Ratio,Cash Flow Margin,Inventory Turnover,Asset Turnover,Receivables Turnover,CapEx Intensity,ROCE,FCF Conversion,Altman Z-Score,PEG Ratio,Price-to-Book
2024-03-31,1,2024,0.260,0.466,0.307,0.070,0.319,1.0371,0.9868,0.2640,3.5477,0.7801,0.250,14.5624,0.2690,4.1559,-0.0220,0.156,1.044,7.8539,NaN,NaN
2024-06-30,2,2024,0.250,0.463,0.296,0.065,0.322,0.9530,0.9061,0.1942,3.9711,0.7988,0.336,13.9135,0.2587,3.7630,-0.0251,0.151,1.446,7.6999,NaN,NaN
2024-09-30,3,2024,0.155,0.462,0.312,0.040,0.259,0.8673,0.8260,0.1698,5.4088,0.8440,0.282,13.0291,0.2601,2.8414,-0.0306,0.181,2.017,6.5961,NaN,NaN
2024-12-31,4,2024,0.292,0.469,0.345,0.106,0.544,0.9229,0.8751,0.2099,4.1542,0.8060,0.241,17.9858,0.3612,4.1938,-0.0237,0.262,0.905,7.5950,NaN,NaN
2025-03-31,1,2025,0.260,0.471,0.310,0.075,0.371,0.8209,0.7775,0.1948,3.9589,0.7983,0.251,15.2112,0.2879,3.6486,-0.0322,0.179,1.091,7.6672,4.2754,47.7974


In [96]:
stock.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p